## TimesFM（timesfm）公開API（関数・クラス）一覧を自動収集

`inspect`（内省）と `pkgutil`（パッケージ走査）で、`timesfm` パッケージの公開API（関数・クラス）を列挙し、`pandas.DataFrame` に格納して表示します。

- 収集対象：トップレベル + サブモジュール（importできる範囲）
- import失敗（オプション依存など）はログとして別途確認可能


In [1]:
%%bash
python "C:/moinfo/libs/timesfm/03_scripts/introspect_timesfm_features.py" \
  --pkg timesfm \
  --out "C:/moinfo/libs/timesfm/04_outputs/api/timesfm_public_api.json" \
  --print


Couldn't find program: 'bash'


## TimesFM（timesfm）公開API（関数・クラス）を内省して DataFrame 化

`inspect`（内省）と `pkgutil`（パッケージ走査）で、`timesfm` の公開API（関数・クラス）を抽出し、`pandas.DataFrame` に格納して表示します。

- 収集対象：トップレベル + import可能なサブモジュール
- import失敗（オプション依存など）は `modules_failed` に記録


In [2]:
import sys
import pandas as pd
from IPython.display import display

# 共通モジュールを import できるようにする
COMMON_DIR = r"C:\moinfo\common"
if COMMON_DIR not in sys.path:
    sys.path.insert(0, COMMON_DIR)

from moinfo_introspect.public_api import collect_public_api, to_dataframe

# 公開APIを収集
data = collect_public_api("timesfm", include_submodules=True)

# DataFrame化
df = to_dataframe(data)

# 概要（メトリクス）
summary_df = pd.DataFrame({
    "metric": ["api_items", "modules_ok", "modules_failed"],
    "value": [len(data["api"]), len(data["modules_ok"]), len(data["modules_failed"])]
})
display(summary_df)

# 種別内訳（class/function）
display(df["kind"].value_counts(dropna=False).to_frame("count"))

# テーブル本体（重いなら df.head(50) に変更）
display(df)


,metric,value
0,api_items,21
1,modules_ok,7
2,modules_failed,4


,count
kind,
class,18
function,3


,qualname,kind,module,signature,doc_first_line
0,timesfm.ForecastConfig,class,timesfm.configs,"(max_context: int = 0, max_horizon: int = 0, n...",Options for forecasting.
1,timesfm.TimesFM_2p5_200M_torch,class,timesfm.timesfm_2p5.timesfm_2p5_torch,"(*args, **kwargs) -> ~T",PyTorch implementation of TimesFM 2.5 with 200...
2,timesfm.configs.ForecastConfig,class,timesfm.configs,"(max_context: int = 0, max_horizon: int = 0, n...",Options for forecasting.
3,timesfm.configs.RandomFourierFeaturesConfig,class,timesfm.configs,"(input_dims: int, output_dims: int, projection...",Framework-agnostic config for random fourier f...
4,timesfm.configs.ResidualBlockConfig,class,timesfm.configs,"(input_dims: int, hidden_dims: int, output_dim...",Framework-agnostic config for a residual block.
5,timesfm.configs.StackedTransformersConfig,class,timesfm.configs,"(num_layers: int, transformer: timesfm.configs...",Framework-agnostic config for a stacked transf...
6,timesfm.configs.TransformerConfig,class,timesfm.configs,"(model_dims: int, hidden_dims: int, num_heads:...",Framework-agnostic config for a transformer.
7,timesfm.torch.dense.RandomFourierFeatures,class,timesfm.torch.dense,(config: timesfm.configs.RandomFourierFeatures...,Random Fourier features layer.
8,timesfm.torch.dense.ResidualBlock,class,timesfm.torch.dense,(config: timesfm.configs.ResidualBlockConfig),Residual block with two linear layers and a li...
9,timesfm.torch.normalization.RMSNorm,class,timesfm.torch.normalization,"(num_features: int, *, epsilon: float = 1e-06)",RMS normalization.


## 生成された JSON を読み込み、DataFrame として可視化

CLIで書き出した `timesfm_public_api.json` を読み込んで表示します（NotebookとCLIの結果一致確認用）。


In [3]:
import json
import pandas as pd
from IPython.display import display

json_path = r"C:\moinfo\libs\timesfm\04_outputs\api\timesfm_public_api.json"

with open(json_path, "r", encoding="utf-8") as f:
    data_from_json = json.load(f)

df2 = pd.DataFrame(data_from_json.get("api", []))

display(pd.DataFrame({
    "metric": ["api_items", "modules_ok", "modules_failed"],
    "value": [len(df2), len(data_from_json.get("modules_ok", [])), len(data_from_json.get("modules_failed", []))]
}))
display(df2["kind"].value_counts(dropna=False).to_frame("count"))
display(df2)


,metric,value
0,api_items,21
1,modules_ok,7
2,modules_failed,4


,count
kind,
class,18
function,3


,qualname,kind,module,signature,doc_first_line
0,timesfm.ForecastConfig,class,timesfm.configs,"(max_context: int = 0, max_horizon: int = 0, n...",Options for forecasting.
1,timesfm.TimesFM_2p5_200M_torch,class,timesfm.timesfm_2p5.timesfm_2p5_torch,"(*args, **kwargs) -> ~T",PyTorch implementation of TimesFM 2.5 with 200...
2,timesfm.configs.ForecastConfig,class,timesfm.configs,"(max_context: int = 0, max_horizon: int = 0, n...",Options for forecasting.
3,timesfm.configs.RandomFourierFeaturesConfig,class,timesfm.configs,"(input_dims: int, output_dims: int, projection...",Framework-agnostic config for random fourier f...
4,timesfm.configs.ResidualBlockConfig,class,timesfm.configs,"(input_dims: int, hidden_dims: int, output_dim...",Framework-agnostic config for a residual block.
5,timesfm.configs.StackedTransformersConfig,class,timesfm.configs,"(num_layers: int, transformer: timesfm.configs...",Framework-agnostic config for a stacked transf...
6,timesfm.configs.TransformerConfig,class,timesfm.configs,"(model_dims: int, hidden_dims: int, num_heads:...",Framework-agnostic config for a transformer.
7,timesfm.torch.dense.RandomFourierFeatures,class,timesfm.torch.dense,(config: timesfm.configs.RandomFourierFeatures...,Random Fourier features layer.
8,timesfm.torch.dense.ResidualBlock,class,timesfm.torch.dense,(config: timesfm.configs.ResidualBlockConfig),Residual block with two linear layers and a li...
9,timesfm.torch.normalization.RMSNorm,class,timesfm.torch.normalization,"(num_features: int, *, epsilon: float = 1e-06)",RMS normalization.


# TimesFM 機能（公開API）イントロスペクト

このノートは `timesfm` パッケージの関数・クラス・メソッドを機械的に列挙し、`pandas.DataFrame` に格納します。  
副作用（import時に重い依存関係がロードされる等）があり得るため、必要なら「サブモジュール走査」をオフにします。


In [4]:
from pathlib import Path
import sys

ROOT = Path(r"C:\moinfo")
SRC_DIR = ROOT / "libs" / "timesfm" / "02_src"

# ローカルモジュール（moinfo_timesfm_ext）を import できるようにする
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

import pandas as pd

# timesfm がインストールされているか確認（失敗したら pip/conda 側の問題）
import importlib
timesfm = importlib.import_module("timesfm")

print("timesfm module:", timesfm.__name__)
print("timesfm file  :", getattr(timesfm, "__file__", "n/a"))
print("python        :", sys.version)


timesfm module: timesfm
timesfm file  : c:\Users\hashimoto.ryohei\miniconda3\envs\kaiseki\Lib\site-packages\timesfm\__init__.py
python        : 3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]


## API 解析（関数・クラス・メソッド一覧）→ DataFrame 化


In [5]:
from pathlib import Path
import sys

ROOT = Path(r"C:\moinfo")
PKG_ROOT = ROOT / "libs" / "timesfm" / "02_src" / "moinfo_timesfm_ext"  # ★ここが重要

if str(PKG_ROOT) not in sys.path:
    sys.path.insert(0, str(PKG_ROOT))

# 動作確認
import moinfo_timesfm_ext
print("moinfo_timesfm_ext:", moinfo_timesfm_ext.__file__)


moinfo_timesfm_ext: C:\moinfo\libs\timesfm\02_src\moinfo_timesfm_ext\moinfo_timesfm_ext\__init__.py


In [7]:
from moinfo_timesfm_ext.moinfo_timesfm_ext.introspect_timesfm import collect_public_api, save_api_df

df = collect_public_api(
    package_name="timesfm",
    include_submodules=True,
    include_private=False,
    include_class_members=True,
    max_doc_chars=250,
)

df.to_csv(r"C:\moinfo\libs\timesfm\04_outputs\tables\timesfm_public_api_from_ext.csv", index=False)


ModuleNotFoundError: No module named 'moinfo_timesfm_ext.moinfo_timesfm_ext'

## TimesFM 公開APIの概要分析・分類・可視化

- 種別（class/function/method/attribute/import_error）の分布
- モジュール階層ごとの規模（どこに機能が集中しているか）
- キーワードベースの機能分類（forecast / config / torch / util など）
- ヒートマップで「種別×分類」の密度を見る
- （任意）docstring を TF-IDF + k-means でクラスタリングして特徴語を抽出


In [8]:
# 任意：doc_summary を TF-IDF + k-means でクラスタリング
# 目的：ルール分類では拾いきれない「自然な機能グループ」を見つける

try:
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.cluster import KMeans

    texts = df2["doc_summary"].fillna("").astype(str)
    # 空文は落とす（クラスタリングのノイズになる）
    mask = texts.str.len() > 20
    texts2 = texts[mask]
    df_text = df2.loc[mask].copy()

    vectorizer = TfidfVectorizer(
        lowercase=True,
        stop_words="english",
        max_features=5000,
        ngram_range=(1, 2),
    )
    X = vectorizer.fit_transform(texts2)

    k = 8  # ざっくり。増やすと細かく割れる
    km = KMeans(n_clusters=k, n_init=10, random_state=0)
    labels = km.fit_predict(X)
    df_text["cluster"] = labels

    # クラスタごとの代表語（上位）
    terms = np.array(vectorizer.get_feature_names_out())
    top_words = []
    for i in range(k):
        center = km.cluster_centers_[i]
        top_idx = center.argsort()[::-1][:12]
        top_words.append(", ".join(terms[top_idx]))

    cluster_sizes = df_text["cluster"].value_counts().sort_index()
    summary = pd.DataFrame({
        "cluster": range(k),
        "size": [cluster_sizes.get(i, 0) for i in range(k)],
        "top_terms": top_words,
    }).sort_values("size", ascending=False)

    display(summary)

    # 可視化：クラスタサイズ
    plt.figure()
    plt.bar(summary["cluster"].astype(str), summary["size"])
    plt.title("Docstring clusters (k-means) sizes")
    plt.tight_layout()
    plt.savefig(OUT_FIG / "timesfm_api_doc_clusters_sizes.png", dpi=200)
    plt.show()

    # 保存
    summary.to_csv(OUT_TBL / "timesfm_api_doc_clusters_summary.csv", index=False, encoding="utf-8-sig")
    df_text.to_csv(OUT_TBL / "timesfm_api_with_doc_clusters.csv", index=False, encoding="utf-8-sig")

    print("Saved cluster outputs to:", OUT_TBL)

except Exception as e:
    print("Clustering skipped (scikit-learn not available or other error):", repr(e))


Clustering skipped (scikit-learn not available or other error): KeyError('doc_summary')


In [9]:
import plotly.express as px

# 1) kind 分布（class/method/function...）
kind_counts = df2["kind"].value_counts().reset_index()
kind_counts.columns = ["kind", "count"]

fig = px.bar(kind_counts, x="kind", y="count", title="TimesFM API: kind distribution")
fig.update_layout(xaxis_tickangle=-45)
fig.show()
fig.write_html(str(OUT_FIG / "timesfm_kind_distribution.html"))

# 2) category 分布（ルール分類）
cat_counts = df2["category"].value_counts().reset_index()
cat_counts.columns = ["category", "count"]

fig = px.bar(cat_counts, x="category", y="count", title="TimesFM API: category distribution (rule-based)")
fig.update_layout(xaxis_tickangle=-45)
fig.show()
fig.write_html(str(OUT_FIG / "timesfm_category_distribution.html"))

# 3) 上位モジュール（どこに機能が集中してる？）
top_modules = (
    df2.groupby("module_l2")
       .size()
       .sort_values(ascending=False)
       .head(25)
       .reset_index(name="count")
)

fig = px.bar(top_modules[::-1], x="count", y="module_l2", orientation="h",
             title="Top modules (level2) by API surface")
fig.show()
fig.write_html(str(OUT_FIG / "timesfm_top_modules_l2.html"))

# 4) ヒートマップ：category × kind
pivot = df2.pivot_table(index="category", columns="kind", values="qualname", aggfunc="count", fill_value=0)
pivot = pivot.loc[pivot.sum(axis=1).sort_values(ascending=False).index, :]  # 行を多い順

fig = px.imshow(
    pivot,
    title="Heatmap: category × kind (counts)",
    labels=dict(x="kind", y="category", color="count"),
    aspect="auto",
)
fig.show()
fig.write_html(str(OUT_FIG / "timesfm_heatmap_category_kind.html"))

print("Saved HTML figures to:", OUT_FIG)


c:\Users\hashimoto.ryohei\miniconda3\envs\kaiseki\Lib\site-packages\kaleido\__init__.py:14: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




NameError: name 'OUT_FIG' is not defined

In [ ]:
from __future__ import annotations

import re
from pathlib import Path
import pandas as pd

ROOT = Path(r"C:\moinfo")
OUT_TBL = ROOT / "libs" / "timesfm" / "04_outputs" / "tables"
OUT_TBL.mkdir(parents=True, exist_ok=True)

# --- どのDFに "cluster" が入っているか吸収（df_text があれば優先）---
df_clustered = None
if "df_text" in globals() and isinstance(df_text, pd.DataFrame) and "cluster" in df_text.columns:
    df_clustered = df_text.copy()
elif "df2" in globals() and isinstance(df2, pd.DataFrame) and "cluster" in df2.columns:
    df_clustered = df2.copy()
elif "df" in globals() and isinstance(df, pd.DataFrame) and "cluster" in df.columns:
    df_clustered = df.copy()

if df_clustered is None:
    raise RuntimeError("cluster 列が見つかりません。先に TF-IDF+kmeans のセルを実行して df_text を作ってください。")

# --- summary（cluster/size/top_terms）がある前提 ---
if "summary" not in globals():
    raise RuntimeError("summary(DataFrame: cluster/size/top_terms) が見つかりません。先にクラスタ要約を作ってください。")

# --- top_terms からテーマ推定（“テーブルから取得”のため、top_terms依存で判定）---
def infer_theme(top_terms: str) -> tuple[str, str]:
    t = (top_terms or "").lower()

    if re.search(r"\bcompile\b|share_mem|torch", t):
        return ("Torch統合・高速化", "モデルのコンパイル(compile)やTorch実行系（高速デコード等）")
    if re.search(r"state_dict|hook", t):
        return ("状態管理・フック", "重み(state_dict)の保存/読込、Hook(フック)で挙動を差し込む")
    if re.search(r"yields|iterator|return iterator", t):
        return ("探索・列挙", "モジュール/サブモジュール/パラメータをイテレータで列挙する")
    if re.search(r"parameters|buffers", t):
        return ("パラメータ・バッファ", "parameters/buffers の取得・操作（学習/推論の材料）")
    if re.search(r"modifies|module place|in place", t):
        return ("in-place変換", "dtype/device 変更など“その場で”モジュールを書き換える操作")
    if re.search(r"target exists|exists|error", t):
        return ("検証・例外", "存在チェックやエラー処理（主にユーティリティ/検証）")
    if re.search(r"evaluation mode|evaluation|mode", t):
        return ("モード切替", "train/eval など推論モード/学習モードの切り替え")
    if re.search(r"child|accessed|using given|get_", t):
        return ("階層アクセス", "子モジュール/特定パラメータへ名前でアクセス（get_submodule等）")

    return ("その他", "雑多な補助機能")

# --- 各クラスタの代表API（qualname）を抽出 ---
# 代表の選び方：doc_summaryが長い＝説明がありそう、さらに timesfm/forecast/compile を優先
def pick_examples(g: pd.DataFrame, k: int = 6) -> pd.DataFrame:
    g = g.copy()
    txt = (g["qualname"].fillna("") + " " + g["doc_summary"].fillna("")).str.lower()
    score = (
        g["doc_summary"].fillna("").str.len()
        + txt.str.contains("forecast").astype(int) * 200
        + txt.str.contains("compile").astype(int) * 150
        + txt.str.contains("timesfm").astype(int) * 80
    )
    g["_score"] = score
    return g.sort_values("_score", ascending=False).head(k)

examples = (
    df_clustered.groupby("cluster", group_keys=False)
    .apply(pick_examples, k=6)
    .loc[:, ["cluster", "qualname", "kind", "module", "doc_summary"]]
)

# --- 概要テーブル作成 ---
ov = summary.copy()
ov["theme"], ov["what_you_can_do"] = zip(*ov["top_terms"].apply(infer_theme))

# 代表APIを文字列にまとめる
ex_str = (
    examples.assign(ex=lambda d: d["qualname"] + " [" + d["kind"].astype(str) + "]")
    .groupby("cluster")["ex"]
    .apply(lambda s: "\n".join(s.tolist()))
    .reset_index()
    .rename(columns={"ex": "representative_api"})
)

overview = ov.merge(ex_str, on="cluster", how="left").sort_values("size", ascending=False)

display(overview)

# 保存（フルパス）
out_csv = OUT_TBL / "timesfm_cluster_overview.csv"
overview.to_csv(out_csv, index=False, encoding="utf-8-sig")
print("saved:", out_csv)


In [ ]:
from __future__ import annotations

from pathlib import Path
import re

from IPython.display import Markdown, display

ROOT = Path(r"C:\moinfo")
OUT_FIG = ROOT / "libs" / "timesfm" / "04_outputs" / "figs"
OUT_FIG.mkdir(parents=True, exist_ok=True)

# Mermaidで壊れやすい文字を軽くエスケープ
def esc(s: str) -> str:
    s = (s or "")
    s = s.replace('"', "'")
    s = re.sub(r"[\r\n]+", " ", s)
    return s

lines = []
lines.append("graph TD")
lines.append('  R["timesfm API"]')

# overview がある前提
for _, r in overview.iterrows():
    c = int(r["cluster"])
    theme = esc(r["theme"])
    size = int(r["size"])
    node_c = f"C{c}"
    lines.append(f'  R --> {node_c}["C{c}: {theme}\\n(size={size})"]')

    reps = (r.get("representative_api") or "").splitlines()
    reps = reps[:6]  # 多すぎると見づらいので上限
    for i, api in enumerate(reps, start=1):
        api = esc(api)
        node_a = f"{node_c}_{i}"
        lines.append(f'  {node_c} --> {node_a}["{api}"]')

mermaid_code = "\n".join(lines)

# 1) Notebook上に表示（Mermaid対応ビューアがある環境ならそのまま描画される）
display(Markdown("```mermaid\n" + mermaid_code + "\n```"))

# 2) ファイル保存（フルパス）
out_mmd = OUT_FIG / "timesfm_api_clusters.mmd"
out_md  = OUT_FIG / "timesfm_api_clusters_mermaid.md"
out_mmd.write_text(mermaid_code, encoding="utf-8")
out_md.write_text("```mermaid\n" + mermaid_code + "\n```", encoding="utf-8")

print("saved:", out_mmd)
print("saved:", out_md)


# 1. サンプルデータセットの作成
TimesFMの機能検証を行うため、明確な**トレンド（傾向）**と**季節性（周期性）**を持つ人工的な時系列データを生成します。

モデルのコンテキスト長（入力）と予測ホライゾン（出力）をカバーできる十分な長さのデータを、以下の仕様で作成します。

* **データ構造**: Pandas DataFrame (ロング形式)
    * `date`: 日時インデックス
    * `unique_id`: 系列を識別するID（多変量/複数系列対応のため）
    * `value`: 観測値
* **データ内容**:
    * `series_01`: サイン波 + 上昇トレンド（基本的な周期的変動）
    * `series_02`: コサイン波 + 緩やかなトレンド（位相の異なる変動）

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def generate_synthetic_series(length=1000, start_date="2024-01-01", freq="h"):
    """
    トレンド、季節性、ノイズを含む人工的な時系列データを生成する関数
    """
    # 日時インデックスの作成
    dates = pd.date_range(start=start_date, periods=length, freq=freq)
    x = np.linspace(0, 8 * np.pi, length)

    # 系列1: サイン波 + 強いトレンド
    y1 = np.sin(x) + np.linspace(0, 5, length) + np.random.normal(0, 0.1, length)
    df1 = pd.DataFrame({
        "unique_id": "series_01",
        "date": dates,
        "value": y1
    })

    # 系列2: コサイン波 + 弱いトレンド + 異なるノイズ
    y2 = np.cos(x) + np.linspace(0, 2, length) + np.random.normal(0, 0.15, length)
    df2 = pd.DataFrame({
        "unique_id": "series_02",
        "date": dates,
        "value": y2
    })

    # 結合してロング形式へ
    df_combined = pd.concat([df1, df2], ignore_index=True)
    return df_combined

# データ生成（TimesFMのデフォルトコンテキスト長 512 + 予測 128 をカバーできるサイズ）
DATA_LENGTH = 1024
df_sample = generate_synthetic_series(length=DATA_LENGTH)

# データの確認
print(f"Data Shape: {df_sample.shape}")
display(df_sample.head())

# 可視化
plt.figure(figsize=(12, 5))
for uid, group in df_sample.groupby("unique_id"):
    plt.plot(group["date"], group["value"], label=uid, alpha=0.8)
    
plt.title("Synthetic Time Series Data for TimesFM Testing")
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# 2. モデルのロード (修正版)

`timesfm` ライブラリの仕様に合わせて、クラス名を `TimesFm` (mは小文字) に修正し、パラメータを `TimesFmHparams` 経由で設定します。

**主な修正点:**
1. クラス名: `TimesFM` -> `TimesFm`
2. パラメータ指定: `hparams` 引数に `TimesFmHparams` オブジェクトを渡す形式に変更
3. チェックポイント指定: `checkpoint` 引数に `TimesFmCheckpoint` オブジェクトを渡す形式に変更

In [11]:
import timesfm
import numpy as np

# 1. 予測設定 (ForecastConfig) の定義
# 実際の入力長や予測したい期間に合わせて設定します
forecast_config = timesfm.configs.ForecastConfig(
    max_context=512,       # 入力系列の最大長
    max_horizon=128,       # 予測期間の最大長
    per_core_batch_size=1  # 1回の推論で処理するバッチサイズ
)

# 2. モデルのコンパイル (必須手順)
print("Compiling model (this may take a few seconds)...")
tfm.compile(forecast_config)
print("Model compiled successfully.")

# 3. 動作確認: ダミーデータによる予測
# サイン波のダミーデータを作成 (長さ512)
dummy_input = [np.sin(np.linspace(0, 20, 512))]

# 予測実行
# 戻り値: (点予測の結果, その他の情報) のタプル
forecast_result = tfm.forecast(horizon=128, inputs=dummy_input)

# 結果の確認
# forecast_result[0] が予測値のメイン配列です
print(f"Forecast result type: {type(forecast_result)}")
print(f"Point forecast shape: {forecast_result[0].shape}")  # (Batch, Horizon) -> (1, 128) expected

# 簡易プロット (オプション)
import matplotlib.pyplot as plt
plt.plot(np.arange(512), dummy_input[0], label="History")
plt.plot(np.arange(512, 512+128), forecast_result[0][0], label="Forecast")
plt.legend()
plt.show()

Compiling model (this may take a few seconds)...


NameError: name 'tfm' is not defined

# 2-1. モデルロード方法の比較検証

ご要望の通り、以下の2つのロード方法を両方試して検証します。

1.  **`from_pretrained()`**: Hugging Face Hub からモデル定義と重みを一括でロードする（標準的方法）。
2.  **`load_checkpoint()`**: ローカルに保存された重みファイルを、初期化済みのインスタンスに読み込む。

**検証フロー:**
1.  `from_pretrained` でモデル (A) をロード。
2.  モデル (A) をローカルディレクトリに保存 (`save_pretrained`)。
3.  新しい空のモデルインスタンス (B) を作成。
4.  モデル (B) に `load_checkpoint` を使用して、保存したローカルの重みを読み込む。

In [ ]:
import timesfm
import torch
import os

# デバイス設定
device = "cuda" if torch.cuda.is_available() else "cpu"

# --- 準備: 元モデルのロード ---
print("=== Loading Source Model ===")
tfm_hub = timesfm.TimesFM_2p5_200M_torch.from_pretrained(
    "google/timesfm-2.5-200m-pytorch",
    backend=device
)

# =========================================================
# アプローチ1: 正攻法 (save_pretrained -> from_pretrained)
# =========================================================
print("\n=== Approach 1: save_pretrained -> from_pretrained (Recommended) ===")
save_dir = "./timesfm_hf_local"
tfm_hub.save_pretrained(save_dir)
print(f"Model saved to (HF format): {save_dir}")

# ローカルパスを指定してロード
try:
    tfm_local_hf = timesfm.TimesFM_2p5_200M_torch.from_pretrained(save_dir, backend=device)
    print("Success: Loaded locally using from_pretrained()!")
except Exception as e:
    print(f"Failed: {e}")

# =========================================================
# アプローチ2: load_checkpoint の検証 (torch.save -> load_checkpoint)
# =========================================================
print("\n=== Approach 2: torch.save -> load_checkpoint (Low-level) ===")
pt_file = "./timesfm_state_dict.pt"

# 1. 生の state_dict を .pt ファイルとして保存
# (Hugging Faceの形式ではなく、PyTorch標準の単一ファイル形式)
torch.save(tfm_hub.model.state_dict(), pt_file)
print(f"Model weights saved to (PyTorch format): {pt_file}")

# 2. 空のモデルを作成して load_checkpoint で読み込む
tfm_local_pt = timesfm.TimesFM_2p5_200M_torch(backend=device)

try:
    # load_checkpoint は内部で torch.load を呼んでいる可能性が高い
    tfm_local_pt.load_checkpoint(pt_file)
    print("Success: Loaded locally using load_checkpoint()!")
    
    # コンパイル確認
    print("Compiling loaded model...")
    tfm_local_pt.compile(timesfm.configs.ForecastConfig(max_context=64, max_horizon=32))
    print("Compilation successful.")
    
except Exception as e:
    print(f"Failed to load checkpoint: {e}")
    # 失敗した場合、モデルの構造(キー名)を確認するデバッグ情報を出す
    try:
        loaded_dict = torch.load(pt_file)
        print("Debug: Keys in saved file (top 5):", list(loaded_dict.keys())[:5])
    except:
        pass

# 3. 予測設定 (Forecast Configuration)

モデルのコンパイルを行う前に、予測時の挙動を制御する **`ForecastConfig`** オブジェクトを作成します。
ここで設定するパラメータは、メモリ使用量や推論速度、予測の精度に影響します。

主な設定項目:
* `max_context`: モデルに入力する過去データの最大長（これより長い系列は切り詰められます）。
* `max_horizon`: 一度に予測する未来の期間（予測ホライゾン）。
* `per_core_batch_size`: 1回の推論処理でまとめて処理する系列の数（バッチサイズ）。
* `normalize_inputs`: 入力データを正規化するかどうか（デフォルトは `False` ですが、データによっては有効）。
* `use_continuous_quantile_head`: 分位点予測（不確実性の推定）を行う場合の設定。

In [ ]:
import timesfm

# 予測設定の定義
# API定義に基づき、timesfm.configs.ForecastConfig を使用します
forecast_config = timesfm.configs.ForecastConfig(
    max_context=512,       # 入力コンテキスト長 (例: 過去512ステップ)
    max_horizon=128,       # 予測ホライゾン (例: 未来128ステップ)
    per_core_batch_size=32,# バッチサイズ (GPUメモリに応じて調整)
    
    # その他のオプション設定 (必要に応じて有効化)
    normalize_inputs=True, # 入力系列の正規化を行うか
    # quantiles=[0.1, 0.5, 0.9] # 分位点予測が必要な場合 (モデルの対応状況による)
)

# 設定内容の確認
print("Forecast Config created:")
print(f"  Max Context: {forecast_config.max_context}")
print(f"  Max Horizon: {forecast_config.max_horizon}")
print(f"  Batch Size : {forecast_config.per_core_batch_size}")
print(f"  Normalize  : {forecast_config.normalize_inputs}")

# 3-1. ForecastConfig 引数の網羅的確認

`ForecastConfig` で設定可能なすべての引数を確認します。
解析結果およびライブラリの仕様に基づく設定項目は以下の通りです。

| 引数名 | 型 | デフォルト値 | 概要 (推定) |
| :--- | :--- | :--- | :--- |
| **`max_context`** | `int` | `0` (必須) | 入力系列の最大長。これを超える過去データは切り捨てられます。 |
| **`max_horizon`** | `int` | `0` (必須) | 予測する未来の期間（予測ホライゾン）の最大長。 |
| **`per_core_batch_size`** | `int` | `1` | 1回の推論ステップで処理する系列数（バッチサイズ）。 |
| **`normalize_inputs`** | `bool` | `False` | 入力系列をモデルに入力する前に正規化（平均0、分散1など）するかどうか。 |
| **`window_size`** | `int` | `0` | (高度な設定) 特定のウィンドウ処理を行う場合のサイズ。 |
| **`use_continuous_quantile_head`** | `bool` | `False` | 連続的な分位点予測ヘッドを使用するかどうか（不確実性予測用）。 |
| **`force_flip_invariance`** | `bool` | `True` | 上下反転に対する不変性を強制するかどうか。 |
| **`infer_is_positive`** | `bool` | `True` | 値が正であることを推論時に考慮するかどうか。 |
| **`fix_quantile_crossing`** | `bool` | `False` | 分位点同士の交差（矛盾）を修正するかどうか。 |
| **`return_backcast`** | `bool` | `False` | 予測だけでなく、入力期間の適合値（バックキャスト）も返すかどうか。 |

以下のコードで、実際のライブラリ定義からドキュメントとシグネチャを出力して確認します。

In [ ]:
import timesfm
import inspect

# ForecastConfig クラスのシグネチャ（引数定義）を取得して表示
print("--- ForecastConfig Signature ---")
sig = inspect.signature(timesfm.configs.ForecastConfig)
for name, param in sig.parameters.items():
    print(f"{name}: {param.annotation} = {param.default}")

print("\n--- Docstring ---")
print(timesfm.configs.ForecastConfig.__doc__)

# 3-2. 設定パラメータの一覧化と網羅的検証

`ForecastConfig` に設定可能なすべての引数をカラム（列）として持ち、検証したい値を格納した一覧表（DataFrame）を作成します。
この表データを基に、実際に `ForecastConfig` オブジェクトを生成し、すべての値が正しく反映されるか検証します。

**検証パターン:**
1. **Case 0 (Standard)**: 一般的な推奨設定（正規化ON、非負制約ONなど）。
2. **Case 1 (Experimental)**: すべてのオプション機能を有効化または変更した設定（分位点ヘッド使用、バックキャスト取得など）。

In [ ]:
import pandas as pd
import timesfm

# 1. 検証用パラメータセットの定義
# 引数名をキー、設定したい値のリストを値とする辞書を作成
# Case 0: 実用的で標準的な設定
# Case 1: オプション機能を網羅的に変更した設定
validation_data = {
    "max_context":                  [512,   1024],
    "max_horizon":                  [128,   256],
    "per_core_batch_size":          [32,    1],
    "normalize_inputs":             [True,  False],
    "infer_is_positive":            [True,  False],
    "force_flip_invariance":        [True,  False],
    "use_continuous_quantile_head": [False, True],
    "fix_quantile_crossing":        [False, True],
    "return_backcast":              [False, True],
    "window_size":                  [0,     10]  # TODO機能だが引数としては存在
}

# 一覧表 (DataFrame) の作成
df_configs = pd.DataFrame(validation_data)
df_configs.index.name = "Case"

print("--- ForecastConfig 検証用パラメータ一覧表 ---")
display(df_configs)

# 2. 網羅的な検証実行
print("\n--- 設定反映の検証結果 ---")

generated_configs = []

for idx, row in df_configs.iterrows():
    # DataFrameの行を辞書に変換
    params = row.to_dict()
    
    # 型の修正 (PandasはIntをFloatにすることがあるため、明示的にintへキャスト)
    int_fields = ["max_context", "max_horizon", "per_core_batch_size", "window_size"]
    for field in int_fields:
        params[field] = int(params[field])
    
    try:
        # Configオブジェクトの生成（引数展開 **params を使用）
        config = timesfm.configs.ForecastConfig(**params)
        
        # 検証ロジック: 入力値とオブジェクトの属性値がすべて一致するか確認
        mismatches = []
        for key, expected_val in params.items():
            actual_val = getattr(config, key)
            if actual_val != expected_val:
                mismatches.append(f"{key}: expected {expected_val}, got {actual_val}")
        
        if not mismatches:
            print(f"[Case {idx}] OK: 全てのパラメータが正常に設定されました。")
            generated_configs.append(config)
        else:
            print(f"[Case {idx}] FAILED: パラメータの不一致があります -> {mismatches}")
            
    except Exception as e:
        print(f"[Case {idx}] ERROR: 設定中にエラーが発生しました -> {e}")

# 最後に、検証に使用する設定（Case 0）を変数に保持
forecast_config = generated_configs[0]
print(f"\n最終的に採用する設定 (Case 0):\n{forecast_config}")

# 3-3. 予測設定クラスのアクセス方法検証

TimesFMのAPIには、トップレベルの `timesfm.ForecastConfig` と、サブモジュールの `timesfm.configs.ForecastConfig` の2つのアクセス経路が存在します。
両方を使用して設定オブジェクトを生成し、動作に違いがないか、およびこれらが同一のクラス定義を指しているかを確認します。

In [ ]:
import timesfm

# テスト用の設定値
test_params = {
    "max_context": 512,
    "max_horizon": 128,
    "per_core_batch_size": 16,
    "normalize_inputs": True
}

print("=== 1. timesfm.ForecastConfig (Top-level) ===")
try:
    config_v1 = timesfm.ForecastConfig(**test_params)
    print("Success: Instance created via timesfm.ForecastConfig")
    print(f"Object type: {type(config_v1)}")
    print(f"Values: context={config_v1.max_context}, horizon={config_v1.max_horizon}")
except AttributeError:
    print("Error: timesfm.ForecastConfig not found.")

print("\n=== 2. timesfm.configs.ForecastConfig (Sub-module) ===")
try:
    config_v2 = timesfm.configs.ForecastConfig(**test_params)
    print("Success: Instance created via timesfm.configs.ForecastConfig")
    print(f"Object type: {type(config_v2)}")
    print(f"Values: context={config_v2.max_context}, horizon={config_v2.max_horizon}")
except AttributeError:
    print("Error: timesfm.configs.ForecastConfig not found.")

print("\n=== 3. Identity Verification ===")
# 両者が同じクラス定義を指しているか確認
try:
    is_same_class = timesfm.ForecastConfig is timesfm.configs.ForecastConfig
    print(f"Are they the same class definition? : {is_same_class}")
    
    # データの内容が一致するか確認 (属性レベル)
    if 'config_v1' in locals() and 'config_v2' in locals():
        # __dict__ を比較して属性が全て同じかチェック
        attrs_match = config_v1.__dict__ == config_v2.__dict__
        print(f"Do the instances have identical attributes? : {attrs_match}")
        
except Exception as e:
    print(f"Comparison failed: {e}")

# 3-4. ForecastConfig クラス定義と引数の完全一致検証

`timesfm.ForecastConfig` と `timesfm.configs.ForecastConfig` が、エイリアス（同一のクラス）であるか、あるいは異なる定義であるかを検証します。
特に「設定できる引数」に差異がないか、`inspect` モジュールを使ってシグネチャレベルで比較します。

In [ ]:
import timesfm
import inspect
import pandas as pd

print("=== ForecastConfig Class Verification ===\n")

# 1. クラスオブジェクトの取得
try:
    cls_top = timesfm.ForecastConfig
    print("Found: timesfm.ForecastConfig")
except AttributeError:
    cls_top = None
    print("Not Found: timesfm.ForecastConfig")

try:
    cls_sub = timesfm.configs.ForecastConfig
    print("Found: timesfm.configs.ForecastConfig")
except AttributeError:
    cls_sub = None
    print("Not Found: timesfm.configs.ForecastConfig")

# 2. 同一クラスかの判定 (Identity Check)
if cls_top and cls_sub:
    is_same = cls_top is cls_sub
    print(f"\n[Identity Check] Are they the same object? : {is_same}")
    if is_same:
        print(">> 結論: 完全に同一のクラスです（トップレベルはエイリアスです）。引数に違いはありません。")
    else:
        print(">> 結論: 異なるクラスオブジェクトです。詳細な引数比較を行います。")

# 3. 引数(シグネチャ)の厳密比較
if cls_top and cls_sub:
    sig_top = inspect.signature(cls_top)
    sig_sub = inspect.signature(cls_sub)
    
    print(f"\n[Signature Check]")
    print(f"Top-level args: {len(sig_top.parameters)} items")
    print(f"Sub-module args: {len(sig_sub.parameters)} items")
    
    # 比較用データフレーム作成
    params_top = {k: str(v) for k, v in sig_top.parameters.items()}
    params_sub = {k: str(v) for k, v in sig_sub.parameters.items()}
    
    # 全ての引数キーを統合
    all_keys = sorted(set(params_top.keys()) | set(params_sub.keys()))
    
    comparison_data = []
    for key in all_keys:
        val_top = params_top.get(key, "(Not Present)")
        val_sub = params_sub.get(key, "(Not Present)")
        match = (val_top == val_sub)
        comparison_data.append([key, val_top, val_sub, match])
        
    df_comparison = pd.DataFrame(comparison_data, columns=["Argument", "timesfm.ForecastConfig", "timesfm.configs.ForecastConfig", "Match"])
    
    # 不一致がある場合のみ表示、または全一致を表示
    if df_comparison["Match"].all():
        print(">> 引数定義は完全に一致しています。")
    else:
        print(">> 差異が見つかりました：")
        display(df_comparison[~df_comparison["Match"]])

    # 全体一覧（確認用）
    print("\n--- 引数一覧 ---")
    display(df_comparison)

elif not cls_top:
    print("\n検証不可: timesfm.ForecastConfig が存在しません。timesfm.configs.ForecastConfig を使用してください。")

# 4. コンパイルと予測実行 (Compile & Forecast)

検証済みの設定 (`forecast_config`) を使用して、以下のフローを実行します。

1.  **コンパイル (`compile`)**: 設定に基づき、モデルの推論プロセスを最適化します（**必須**）。
2.  **データ準備**: サンプル DataFrame (`df_sample`) を、モデルが受け付ける `List[numpy.ndarray]` 形式に変換します。
3.  **予測 (`forecast`)**: コンパイルされたモデルで未来の値を予測します。
4.  **結合・可視化**: 予測結果を元の DataFrame と結合し、グラフで確認します。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- 1. モデルのコンパイル (必須) ---
# 前のステップで作成した推奨設定 (Case 0) を使用
print(f"Compiling model with config: {forecast_config} ...")
tfm.compile(forecast_config)
print("Model compiled successfully.")

# --- 2. データ準備 (DataFrame -> List[np.array]) ---
# TimesFM v2.5 の forecast メソッドはリスト形式の numpy 配列を受け取ります
# unique_id ごとにデータを分割してリスト化
input_data_map = {}
input_arrays = []

# 系列IDの順序を保持するためにリストを作成
unique_ids = df_sample['unique_id'].unique()

for uid in unique_ids:
    # 各系列の値を抽出
    series_values = df_sample[df_sample['unique_id'] == uid]['value'].values
    input_arrays.append(series_values)
    input_data_map[uid] = series_values

print(f"Prepared {len(input_arrays)} input series for forecasting.")

# --- 3. 予測実行 (Forecast) ---
# horizon は config の max_horizon と同じか、それ以下である必要があります
forecast_horizon = forecast_config.max_horizon
print(f"Forecasting next {forecast_horizon} steps...")

# 戻り値: (点予測[Batch, Horizon], その他情報)
forecast_result = tfm.forecast(
    inputs=input_arrays,
    horizon=forecast_horizon
)

# 予測値の取得 (0番目の要素が点予測)
point_forecasts = forecast_result[0]
print(f"Forecast shape: {point_forecasts.shape}") # (2, 128) expected

# --- 4. 結果の整理と可視化 ---
# 予測結果をDataFrame化して可視化
plt.figure(figsize=(15, 6))

for i, uid in enumerate(unique_ids):
    history = input_arrays[i]
    forecast = point_forecasts[i]
    
    # 日時軸の作成 (簡易的に履歴の続きとして生成)
    last_date = df_sample[df_sample['unique_id'] == uid]['date'].max()
    freq = "h" # サンプルデータの頻度
    future_dates = pd.date_range(start=last_date + pd.Timedelta(hours=1), periods=forecast_horizon, freq=freq)
    
    # 履歴のプロット (直近の一部のみ表示)
    display_len = 200
    plt.plot(
        df_sample[df_sample['unique_id'] == uid]['date'].iloc[-display_len:], 
        history[-display_len:], 
        label=f"{uid} (History)", 
        linestyle="--"
    )
    
    # 予測のプロット
    plt.plot(future_dates, forecast, label=f"{uid} (Forecast)", linewidth=2)

plt.title("TimesFM Forecast Result")
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# 5. 単変量予測の比較検証 (Univariate Forecasting)

`forecast()` と `forecast_on_df()` の両方のアプローチを比較します。

1.  **`forecast()`**:
    * **概要**: NumPy配列のリストを入力とする基本API。
    * **特徴**: 高速でオーバーヘッドが少ないが、日付やIDの管理は自分で行う必要がある。
    * **現状**: `TimesFM_2p5_200M_torch` クラス標準実装。

2.  **`forecast_on_df()` (再現実装)**:
    * **概要**: Pandas DataFrame (Long形式) を直接受け取り、予測結果もDataFrameで返す。
    * **特徴**: 前処理・後処理（日付生成、ID結合）を自動化し、分析フローに組み込みやすい。
    * **現状**: v2.5のモデルクラスには直接実装されていないため、ヘルパー関数として定義して使用する。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- A. 標準機能: forecast() の実行 ---
print("=== A. Testing forecast() (Array Input) ===")

# 1. データの準備 (DataFrame -> List[np.ndarray])
# 各系列の値をリストに格納
inputs_list = []
ids = df_sample['unique_id'].unique()
for uid in ids:
    series = df_sample[df_sample['unique_id'] == uid]['value'].values
    inputs_list.append(series)

# 2. 予測実行
# コンパイル済みの設定 (max_horizon=128) を使用
print(f"Input type: List of {len(inputs_list)} arrays")
raw_forecast = tfm.forecast(inputs=inputs_list, horizon=128)

# 結果は (PointForecast, ...) のタプル
point_forecast_array = raw_forecast[0]
print(f"Output shape: {point_forecast_array.shape}")
print("forecast() test passed.\n")


# --- B. 拡張機能: forecast_on_df() の再現と実行 ---
print("=== B. Testing forecast_on_df() (DataFrame Input) ===")

def forecast_on_df(model, df, unique_id_col="unique_id", value_col="value", date_col="date", freq="h"):
    """
    TimesFM v2.5向けに forecast_on_df の機能を再現するラッパー関数
    """
    # 1. 前処理: データフレームから配列リストへ変換
    unique_ids = df[unique_id_col].unique()
    inputs = []
    last_dates = []
    
    for uid in unique_ids:
        sub_df = df[df[unique_id_col] == uid].sort_values(date_col)
        inputs.append(sub_df[value_col].values)
        last_dates.append(sub_df[date_col].max())
    
    # 2. 推論実行
    # configから予測期間を取得
    horizon = model.forecast_config.max_horizon
    forecast_tuple = model.forecast(inputs=inputs, horizon=horizon)
    forecast_values = forecast_tuple[0] # (N, Horizon)
    
    # 3. 後処理: 結果をDataFrame形式に復元
    results = []
    for idx, uid in enumerate(unique_ids):
        # 未来の日付インデックスを作成
        start_date = last_dates[idx] + pd.Timedelta(1, unit=freq)
        future_dates = pd.date_range(start=start_date, periods=horizon, freq=freq)
        
        # 結果用DF作成
        res_df = pd.DataFrame({
            unique_id_col: uid,
            date_col: future_dates,
            "timesfm_forecast": forecast_values[idx]
        })
        results.append(res_df)
        
    return pd.concat(results, ignore_index=True)

# 実行テスト
df_forecast_result = forecast_on_df(
    model=tfm, 
    df=df_sample, 
    unique_id_col="unique_id", 
    value_col="value", 
    date_col="date",
    freq="h"
)

# 結果確認
print("Forecast DataFrame Head:")
display(df_forecast_result.head())

# 可視化 (forecast_on_df の結果を使用)
plt.figure(figsize=(12, 5))
for uid in ids:
    # 実績
    history = df_sample[df_sample['unique_id'] == uid]
    plt.plot(history['date'].iloc[-100:], history['value'].iloc[-100:], label=f"{uid} History", linestyle="--", alpha=0.7)
    
    # 予測
    pred = df_forecast_result[df_forecast_result['unique_id'] == uid]
    plt.plot(pred['date'], pred['timesfm_forecast'], label=f"{uid} Forecast", linewidth=2)

plt.title("Forecast Result using 'forecast_on_df' wrapper")
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# 6. 外部変数付き予測 (再修正版: Backcast有効化)

エラー `ValueError: For XReg, return_backcast must be set to True` に対処するため、予測設定を更新してモデルを再コンパイルします。
XReg（外部変数回帰機能）は、モデルの基本予測と実際の値とのズレ（残差）を外部変数で説明しようとするため、バックキャスト（過去の予測値）が必要です。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import timesfm

# --- 1. 設定の更新と再コンパイル (必須) ---
print("=== Re-Compiling Model with return_backcast=True ===")

# 既存の設定を引き継ぎつつ、return_backcast=True を追加
forecast_config_cov = timesfm.configs.ForecastConfig(
    max_context=512,
    max_horizon=128,
    per_core_batch_size=32,
    normalize_inputs=True,
    
    # 【重要】 共変量(XReg)を使用するための必須設定
    return_backcast=True 
)

# 再コンパイル
tfm.compile(forecast_config_cov)
print("Model re-compiled successfully.")

# --- 2. データ準備 (前回と同様) ---
# 入力系列
input_arrays = [
    df_sample[df_sample['unique_id'] == 'series_01']['value'].values,
    df_sample[df_sample['unique_id'] == 'series_02']['value'].values
]

# 動的共変量 (未来含む): 温度のようなコサイン波
total_len = 512 + 128
t = np.linspace(0, 8 * np.pi, total_len)
dynamic_feature = np.cos(t) * 10 
dynamic_covariates = {
    "temperature": [dynamic_feature, dynamic_feature] 
}

# 静的共変量: カテゴリID
static_covariates = {
    "category_id": [0, 1]
}

# --- 3. 共変量付き予測の実行 ---
print("\n=== Executing forecast_with_covariates() ===")

try:
    forecast_cov_result = tfm.forecast_with_covariates(
        inputs=input_arrays,
        dynamic_numerical_covariates=dynamic_covariates,
        static_categorical_covariates=static_covariates
    )
    
    print("Success: Forecast with covariates completed.")
    point_forecast_cov = forecast_cov_result[0] # (Batch, Horizon)
    print(f"Output shape: {point_forecast_cov.shape}")

except Exception as e:
    print(f"Error: {e}")
    raise

# --- 4. 結果の比較可視化 ---
plt.figure(figsize=(15, 6))

uid_idx = 0 
target_uid = 'series_01'
last_date = df_sample[df_sample['unique_id'] == target_uid]['date'].max()
horizon_len = point_forecast_cov.shape[1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(hours=1), periods=horizon_len, freq="h")

# A. 通常予測 (point_forecasts変数が残っていれば表示)
if 'point_forecasts' in locals():
    # 注意: point_forecastsは再コンパイル前の結果ですが、比較用にプロット
    plt.plot(future_dates, point_forecasts[uid_idx], label="Forecast (No Covariates)", linestyle="--", color="blue", alpha=0.5)

# B. 共変量付き予測
plt.plot(future_dates, point_forecast_cov[uid_idx], label="Forecast (w/ Covariates)", color="red", linewidth=2)

# C. 共変量の可視化 (右軸)
ax2 = plt.gca().twinx()
cov_future = dynamic_covariates["temperature"][uid_idx][-horizon_len:]
ax2.plot(future_dates, cov_future, label="Covariate (Temperature)", color="green", alpha=0.2)
ax2.set_ylabel("Covariate Value")

plt.title(f"Forecast Comparison: With vs Without Covariates ({target_uid})")
plt.xlabel("Date")
plt.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.show()

# 7. ベクトル抽出 (構造特定と自動フック)

モデルの内部構造を表示し、特徴量抽出に最適なレイヤー（通常は最後のTransformerブロックや正規化層）を特定してフックします。

**アプローチ:**
1. `named_modules()` を走査し、モデルの階層構造をテキストで出力して確認します。
2. 名前リストの中から、`layers` や `blocks` を含む最も深いインデックスを持つ層を自動検出し、ターゲットにします。

In [ ]:
import torch
import timesfm
import numpy as np
import matplotlib.pyplot as plt

# --- 1. モデル構造の「レントゲン撮影」 ---
print("=== Inspecting Model Internal Layers ===")

# モデル内の主要なモジュール名を表示 (深さ2まで)
# これで layers, blocks, transformer などの名前を確認します
found_layers = []
for name, module in tfm.model.named_modules():
    # 名前が空（自分自身）はスキップ
    if not name: continue
    
    # 全て表示すると多すぎるので、主要なコンポーネントのみ記録
    found_layers.append(name)
    
    # トップレベル付近のみ表示
    if name.count('.') <= 1:
        print(f" - {name}: {module.__class__.__name__}")

print(f"\nTotal modules found: {len(found_layers)}")

# --- 2. ターゲット層の自動選定 ---
# 「layers.数字」のようなパターンを探し、その最大番号（最終層）を特定します
target_name = None
max_layer_idx = -1

for name in found_layers:
    parts = name.split('.')
    # "layers.19" や "blocks.19" のようなパターンを検出
    for i, part in enumerate(parts):
        if part.isdigit() and i > 0:
            # 数字の前の部分がコンテナ名 (layers, blocks等)
            container_name = parts[i-1]
            idx = int(part)
            
            # コンテナ名が一般的かチェック
            if container_name in ['layers', 'blocks', 'h', 'transformer']:
                if idx > max_layer_idx:
                    max_layer_idx = idx
                    # この数字までのパスをターゲット候補とする
                    target_name = ".".join(parts[:i+1])

# もし数字付きレイヤーが見つからない場合、output_layerやnormを探す
if target_name is None:
    for name in found_layers:
        if 'output' in name or 'final_norm' in name or 'ln_f' in name:
            target_name = name
            break

print(f"\nTarget Layer Auto-Selected: '{target_name}'")

# --- 3. 選択した層へのフックとベクトル抽出 ---
if target_name:
    # 名前から実際のモジュールオブジェクトを取得
    target_module = dict(tfm.model.named_modules())[target_name]
    
    embeddings_storage = {}
    
    # 堅牢なフック関数 (再定義)
    def robust_hook(name):
        def hook(model, input, output):
            # Tensorを探す
            if isinstance(output, torch.Tensor):
                tensor = output
            elif isinstance(output, (tuple, list)):
                tensor = output[0] if len(output) > 0 and isinstance(output[0], torch.Tensor) else None
            else:
                tensor = None
                
            if tensor is not None:
                embeddings_storage[name] = tensor.detach().cpu().numpy()
        return hook

    # フック登録
    handle = target_module.register_forward_hook(robust_hook('feature_embedding'))
    print(f"Hook registered on: {target_module.__class__.__name__}")

    # 予測実行
    print("Running forecast to extract features...")
    try:
        _ = tfm.forecast(inputs=input_data, horizon=128)
    except Exception as e:
        print(f"Forecast warning: {e}")

    # フック解除
    handle.remove()

    # --- 4. 可視化 ---
    if 'feature_embedding' in embeddings_storage:
        emb = embeddings_storage['feature_embedding']
        print(f"\nSuccess! Extracted Embedding Shape: {emb.shape}")
        
        # ヒートマップ
        feature_map = emb[0, :, :200].T
        plt.figure(figsize=(12, 5))
        plt.imshow(feature_map, aspect='auto', cmap='magma', origin='lower')
        plt.colorbar(label='Activation')
        plt.title(f"TimesFM Feature Embeddings\nLayer: {target_name}")
        plt.xlabel("Time Patch Index")
        plt.ylabel("Feature Dimension (First 200)")
        plt.show()
    else:
        print("Error: Failed to capture embeddings (Tensor not found in output).")

else:
    print("Error: Could not automatically identify a target layer.")

# 7-2. 特徴量のテーブル表示 (DataFrame)

モデルの内部構造解析により、`stacked_xf` という名前でTransformerブロックが積み重なっていることが分かりました。
最も抽象度の高い特徴量を持つ **`stacked_xf.19` (最終層)** をターゲットにして再度ベクトル抽出を行い、その数値を**テーブル（DataFrame）**として整形・表示します。

* **行 (Index)**: 時系列のパッチ（時間ステップ）
* **列 (Columns)**: 特徴量の次元（0 ~ 1279）

In [ ]:
import pandas as pd
import torch
import timesfm
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. ターゲット層 (stacked_xf.19) へのフック登録 ---
target_layer_name = 'stacked_xf.19'  # 構造解析で見つかった最終Transformer層
print(f"=== Hooking into target: {target_layer_name} ===")

# モジュールの取得
target_module = dict(tfm.model.named_modules())[target_layer_name]

# データ格納用
embeddings_storage = {}

# 堅牢なフック関数
def robust_hook(name):
    def hook(model, input, output):
        # Tensorを探す (Tupleの中に隠れている場合があるため)
        if isinstance(output, torch.Tensor):
            tensor = output
        elif isinstance(output, (tuple, list)):
            tensor = output[0] if len(output) > 0 and isinstance(output[0], torch.Tensor) else None
        else:
            tensor = None
            
        if tensor is not None:
            embeddings_storage[name] = tensor.detach().cpu().numpy()
    return hook

# フック登録
handle = target_module.register_forward_hook(robust_hook('deep_features'))

# --- 2. 予測実行 (特徴量抽出) ---
print("Running forecast...")
try:
    # バッチサイズ1で実行
    _ = tfm.forecast(inputs=[input_data[0]], horizon=128)
except Exception as e:
    # forecastの戻り値処理でエラーが出ても、フックさえ動けばOK
    print(f"Forecast processed (Warning: {e})")

handle.remove()

# --- 3. テーブル (DataFrame) 化と表示 ---
if 'deep_features' in embeddings_storage:
    # Shape: (Batch, Patch_Length, Dim) -> (1, 16, 1280)
    emb = embeddings_storage['deep_features'][0]  # バッチ先頭を取り出し
    
    # DataFrame作成
    # 行: パッチ(時間), 列: 次元
    df_features = pd.DataFrame(emb)
    df_features.index.name = "Time_Patch_Index"
    df_features.columns.name = "Feature_Dim"
    
    print(f"\nFeature Table Shape: {df_features.shape}")
    print("=== Feature Matrix (Head) ===")
    
    # 大きすぎるので一部を表示 (最初の10列だけなど)
    display(df_features.iloc[:, :10].head())
    
    # 統計情報の確認
    print("\n=== Basic Statistics of Features ===")
    display(df_features.describe().iloc[:, :10]) # 記述統計
    
    # --- (オプション) クラスタリング用データの作成 ---
    # このテーブルを使えば、類似度計算などが可能です
    # 例: パッチごとの相関行列
    corr_matrix = df_features.T.corr()
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(corr_matrix, cmap='coolwarm', center=0)
    plt.title("Time Patch Correlation (Similarity)")
    plt.xlabel("Patch Time")
    plt.ylabel("Patch Time")
    plt.show()

else:
    print("Error: Failed to capture features.")

# 8. 最適化と高速化の検証 (Optimization & Benchmark)

TimesFM v2.5 の `compile()` による最適化効果を検証します。
モデルの計算グラフを事前に構築・最適化することで、2回目以降の推論速度が大幅に向上する（キャッシュ効果）ことを確認します。

**検証項目:**
1.  **Compilation Time**: `tfm.compile()` にかかる時間。
2.  **Warm-up Inference**: コンパイル後、最初の推論にかかる時間（初期化オーバーヘッドを含む）。
3.  **Cached Inference**: 2回目以降の推論にかかる時間（ここが実運用時の速度）。

In [13]:
# --- /full/path/not_applicable_in_notebook (Jupyterセル想定のため) ---
import torch

embeddings_storage = {}

def _first_tensor(x):
    """入れ子構造から最初のtorch.Tensorを取り出す。なければNone。"""
    if isinstance(x, torch.Tensor):
        return x
    if isinstance(x, (tuple, list)):
        for v in x:
            t = _first_tensor(v)
            if t is not None:
                return t
    if isinstance(x, dict):
        for v in x.values():
            t = _first_tensor(v)
            if t is not None:
                return t
    return None

def get_activation(name: str, select=_first_tensor):
    def hook(model, inputs, output):
        t = select(output)
        if t is None:
            return
        embeddings_storage[name] = t.detach().cpu().numpy()
    return hook

